In [ ]:
from Jumpscale import j

j.tools.threebot.init_my_threebot(name='rishi.3bot',email='rv@thesecloud.com',description="Smart Edge Cloud - Estonia")

# Make sure I have an identity (set default one for mainnet of testnet)
me = j.tools.threebot.me.default

print(me)

j.clients.threebot.explorer_addr_set('explorer.grid.tf')

# Load the zero-os sal and reate empty reservation method
zos = j.sal.zosv2
r = zos.reservation_create()

# Set the unique parameters for this network deployment
u_port=int(8101)
u_networkname=str("weynand_new")

print("Port is set to:", u_port)
print("Unique network name is set to:", u_networkname)
# create overlay network definition in datastructure called "network"
network = zos.network.create(r, ip_range="172.20.0.0/16", network_name=u_networkname)

nodes_mazraa = zos.nodes_finder.nodes_search(farm_id=123428) # (IPv6 nodes)
nodes_salzburg = zos.nodes_finder.nodes_search(farm_id=12775) # (IPv6 nodes)
nodes_vienna_1 = zos.nodes_finder.nodes_search(farm_id=82872) # (IPv6 nodes)
nodes_belgium = zos.nodes_finder.nodes_search(farm_id=1) # (IPv4 nodes, to be used as ingress/egress point.  These are not webgatewaysm, just nodes connected to the internet with IPv4 addresses)
nodes_munich = zos.nodes_finder.nodes_search(farm_id=50669) #(IPv6 nodes)

# Make a network spanning Salzburg, Vienna, Lichristu and Munich
nodes_all = nodes_salzburg[:5] + nodes_vienna_1[:5] + nodes_belgium[:5]+nodes_mazraa[:5]


# make sure to set a new port,empty 
for i, node in enumerate(nodes_all):
    if zos.nodes_finder.filter_is_up(node):
        iprange = f"172.20.{i+10}.0/24"
        zos.network.add_node(network, node.node_id , iprange, wg_port=u_port)
        print("Node: ", i, " ID: ", node.node_id, " IPv4 address: ", iprange)

        
# Enter here the node_id for the node that is the IPv4 bridge to create the wireguard config.
wg_config = zos.network.add_access(network, 'CBDY1Fu4CuxGpdU3zLL9QT5DGaRkxjpuJmzV6V5CBWg4', '172.20.100.0/24', ipv4=True)

print("wireguard configuration")
print(wg_config)


# Set the duration for the reservation
import time
expiration = j.data.time.epoch + (10*60)
#expiration = j.data.time.epoch + (5*60)

# register the reservation
rid = zos.reservation_register(r, expiration, identity=me)
time.sleep(5)

# inspect the result of the reservation provisioning
result = zos.reservation_result(rid)
print("provisioning result")
print(result)